In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
paths = [x for x in glob.glob('../input/*/*.csv') if 'amex-default-prediction' not in x]
paths

['../input/amex-base-model/20220816_amex_lightbgm_submission(0.787).csv',
 '../input/amex-base-model/20220820_amex_catboost_submission(0.784).csv',
 '../input/amex-base-model/20220812_amex_xgboost_submission(0.764).csv']

In [3]:
weights = [0.9, 1, 0.8]

In [4]:
dfs = [pd.read_csv(x) for x in paths]
dfs = [x.sort_values(by='customer_ID') for x in dfs]

In [5]:
for df in dfs:
    df['prediction'] = np.clip(df['prediction'], 0, 1)

In [6]:
submit = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')
submit['prediction'] = 0

for df, weight in zip(dfs, weights):
    submit['prediction'] += (df['prediction'] * weight)
    
submit['prediction'] /= np.sum(weights)

submit.to_csv('mean_submission.csv', index=None)

In [7]:
submit = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')
submit['prediction'] = 0

from scipy.stats import rankdata

for df, weight in zip(dfs, weights):
    submit['prediction'] += (rankdata(df['prediction'])/df.shape[0]) * weight
    
submit['prediction'] /= 3

submit.to_csv('submission.csv', index=None)